In [1]:
## Importação das bibliotecas

import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
## Importação dos datasets

df = pd.read_csv(r"C:\TCC\Datasets\data.csv", delimiter = ',', encoding = 'latin-1')

In [3]:
## Elimina espaços em branco antes e depois da string

df.artists.str.strip() 
df.name.str.strip() 


## Dropa colunas não utilizadas

df.drop(columns = ['id', 'release_date'], inplace = True)


## Padroniza as colunas

x = df[df.drop(columns = ['artists', 'name']).columns].values
scaler = StandardScaler().fit(x)
x_scaled = scaler.transform(x)
df[df.drop(columns = ['artists', 'name']).columns] = x_scaled


In [4]:
## Função para buscar as músicas no dataset

def busca_musica(musica, df, number = 10):
    df.drop_duplicates(inplace = True)
    musicas = df['name'].values
    artistas = df['artists'].values
    t = []
    count = 0
    if musica[-1] == ' ':
        musica = musica[:-1]
    for i in musicas:
        if musica.lower() in i.lower():
            t.append([len(musica)/len(i), count])
        else:
            t.append([0, count])
        count += 1
    t.sort(reverse = True)
    s = [[musicas[t[i][1]], artistas[t[i][1]].strip('][').split(', ')] for i in range(number)]   
    lista_musicas = [musicas[t[i][1]] for i in range(number)]
    artista = [artistas[t[i][1]] for i in range(number)]
    x = []
    for i in s:
        l = ''
        by = ''
        for j in i[1]:
            by += j
        l += i[0] + ' por ' + by
        x.append(l)
    tup = []
    for i in range(number):
        tup.append((x[i], i))

    
    return tup, lista_musicas, artista

In [14]:
##Função de recomendação utilizando matriz de correlação

def mat_corr(df, nome_musica, number, artista):
    df.drop_duplicates(subset = ['artists','name'], inplace = True)
    x = df[(df['name'] == nome_musica) & (df['artists'] == artista)].drop(columns = ['name', 'artists']).values
    artista = artista.replace("'", "").replace("'", "").replace('[', '').replace(']', '')
    if ',' in artista:
        inm = artista.rfind(",")
        artista = artista[:inm] + ' e' + artista[inm + 1:]
    print('A música selecionada é:', nome_musica, ' por ', artista, '\n')
    

    song_names = df['name'].values
    p = []
    count = 0
    for i in df.drop(columns = ['artists', 'name']).values:
        p.append([distance.correlation(x, i), count])
        count += 1
    p.sort()
    for i in range(1, number + 1):
        artistas = df['artists'].values
        artista = artistas[p[i][1]]
        artista = artista.replace("'", "").replace("'", "").replace('[', '').replace(']', '')
        if ',' in artista:
            inm = artista.rfind(",")
            artista = artista[:inm] + ' e ' + artista[inm + 1:]
        print(i, '. ', song_names[p[i][1]], 'por', artista)
 

In [15]:
a = input('Insira o nome da música:')
b = int(input('Quantas recomendações você precisa? '))
tup, s, ar = busca_musica(a, df)
ans = widgets.Dropdown(
    options = tup,
    description = 'Selecione:' + a,
    disabled = False,
)
display(ans)

Insira o nome da música:Angel
Quantas recomendações você precisa? 5


Dropdown(description='Selecione:Angel', options=(("Angel por 'Dave Matthews Band'", 0), ("Angel por 'The Marve…

In [16]:
mat_corr(df, s[ans.value], b, ar[ans.value])


A música selecionada é: Angel  por  Dave Matthews Band 

1 .  Every Heart-ãã³ããã­ã¢ã- por BoA
2 .  Feelin' Way Too Damn Good por Nickelback
3 .  I Still Miss You por Keith Anderson
4 .  Roll with Me por Montgomery Gentry
5 .  Water (feat. Rostam) por Ra Ra Riot e  Rostam
